In [ ]:
# https://ai.google.dev/gemini-api/docs/quickstart
# https://ai.google.dev/gemini-api/docs/rate-limits

FREE_MODELS = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "gemma-3-27b-it",
    "gemma-3n-e4b-it",
    "text-embedding-004"
]

LIST_MODELS = False

import httpx
import os
import requests
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pydantic import BaseModel

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

if LIST_MODELS:
    for model in client.models.list():
        print(model.name)

### General

In [ ]:
# Temperature
response = client.models.generate_content(
    model="gemma-3-27b-it",
    config=types.GenerateContentConfig(
        temperature=1.0
    ),
    contents=["Explain how AI works in a few words"]
)
print(response.text)

In [ ]:
# Streaming
response = client.models.generate_content_stream(
    model="gemma-3-27b-it",
    contents=["Explain how AI works in a few words"]
)
for chunk in response:
    print(chunk.text, end="")

In [ ]:
# System instructions
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="You are a cat. Your name is Neko."
    ),
    contents="Explain how AI works in a few words"
)
print(response.text)

In [ ]:
# Chat
chat = client.chats.create(model="gemma-3-27b-it")
response = chat.send_message("I have 2 dogs in my house.")
response = chat.send_message("How many paws are in my house?")

for message in chat.get_history():
    print(f"{message.role}:")
    print(message.parts[0].text)

### Multimodal

In [ ]:
image_path = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"
image_bytes = requests.get(image_path).content
image = types.Part.from_bytes(
  data=image_bytes, mime_type="image/jpeg"
)

response = client.models.generate_content(
    model="gemma-3-27b-it",
    config=types.GenerateContentConfig(
        temperature=1.0
    ),
    contents=[image, "Tell me about this image"]
)
print(response.text)

### Thinking

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Provide a list of 3 famous physicists and their key contributions",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=1024, include_thoughts=True)
        # Turn off thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
        # Turn on dynamic thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=-1)
    )
)
print(response.text)

In [ ]:
for part in response.candidates[0].content.parts:
    if not part.text:
        continue
    if part.thought:
        print("Thought summary:")
        print(part.text)
        print()

### Structured Output

In [ ]:
class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List a few popular cookie recipes, and include the amounts of ingredients.",
    config={
        "response_mime_type": "application/json",
        "response_schema": list[Recipe],
    },
)
print(response.text)

In [ ]:
my_recipes: list[Recipe] = response.parsed
len(my_recipes)

### Function Calling

In [ ]:
weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco",
            },
        },
        "required": ["location"],
    },
}

tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What's the temperature in London?",
    config=config,
)

if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = get_current_temperature(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

In [ ]:
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_weather_forecast(location={location})")
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 25, "unit": "celsius"}

def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature]
)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    config=config,
)
print(response.text)

### Document Understanding

In [ ]:
doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"
doc_data = httpx.get(doc_url).content

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Part.from_bytes(data=doc_data, mime_type='application/pdf'),
        "Summarize this document"
    ]
)
print(response.text)

### Code Execution

In [ ]:
prompt = """\
What is the sum of the first 50 prime numbers? Generate and run code
for the calculation, and make sure you get all 50."""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    )
)
print(response.text)

In [ ]:
for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    if part.executable_code is not None:
        print(part.executable_code.code)
    if part.code_execution_result is not None:
        print(part.code_execution_result.output)

### Search

In [ ]:
tool_config = types.ToolConfig(
    function_calling_config=types.FunctionCallingConfig(
        mode="ANY", allowed_function_names=["get_current_temperature"]
    )
)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Who won the NBA Finals 2025?",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        tool_config=types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(mode="ANY")
        )
    )
)
print(response.text)

In [ ]:
def add_citations(response):
    text = response.text
    supports = response.candidates[0].grounding_metadata.grounding_supports
    chunks = response.candidates[0].grounding_metadata.grounding_chunks

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = ", ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text

text_with_citations = add_citations(response)
print(text_with_citations)

### Embeddings

In [ ]:
result = client.models.embed_content(
    model="text-embedding-004",
    contents=["What is the meaning of life?"]
)
print(result.embeddings)